In [0]:
import json
import requests
import time


dbutils.widgets.text("DASHBOARD_NAME", "")
DASHBOARD_NAME = dbutils.widgets.get("DASHBOARD_NAME")

dbutils.widgets.text("json_file", "")
json_file = dbutils.widgets.get("json_file")

# --- Configuration ---
DASHBOARD_NAME = f"{DASHBOARD_NAME}"                             
WAREHOUSE_ID = "0419731c9b641b8d"                        

# --- Setup ---
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host = ctx.apiUrl().get()
token = ctx.apiToken().get()
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

file_path = f"/Volumes/dbx_migration_poc/dbx_migration_ts/lvdash_files_ak_out/{json_file}"
print(f"Authenticated to {host}")

# -- Delete the Workspace File (Node) ---
# This is the file that causes the "Node... already exists" error.
# We will now try to delete it from multiple common locations.
try:
    user_name = ctx.userName().get()
    
    # List of paths to try deleting
    paths_to_delete = [
        # The user's root directory (the new likely culprit)
        f"/Users/{user_name}/{DASHBOARD_NAME}.lvdash.json", 
        # The default dashboards directory (what we checked before)
        f"/Users/{user_name}/databricks/dashboards/{DASHBOARD_NAME}.lvdash.json"
    ]
    
    print("Attempting to delete conflicting workspace files...")
    
    delete_file_url = f"{host}/api/2.0/workspace/delete"
    file_deleted = False

    for path in paths_to_delete:
        print(f"  > Checking for file at: {path}")
        delete_file_payload = {"path": path}
        
        # Use POST for workspace delete
        delete_file_resp = requests.post(delete_file_url, headers=headers, json=delete_file_payload)

        if delete_file_resp.status_code == 200:
            print(f"  > SUCCESS: Deleted conflicting file at {path}")
            file_deleted = True
            # We found and deleted it, we can stop checking
            break 
        elif delete_file_resp.status_code == 404:
            print(f"  > Info: No file found at {path}.")
        else:
            print(f"  > Warning: Could not delete file at {path}: {delete_file_resp.status_code} - {delete_file_resp.text}")

    if not file_deleted:
        print("No conflicting workspace files were found or deleted.")
        
except Exception as e:
    print(f"Error trying to get username or delete workspace file: {e}. Proceeding anyway...")

print("---")

# --- Delete the Registered Lakeview Dashboard (Robustness) ---
print(f"Checking for existing *registered* dashboard named '{DASHBOARD_NAME}'...")
list_url = f"{host}/api/2.0/lakeview/dashboards"

existing_dashboard = None
list_resp = requests.get(list_url, headers=headers)

if list_resp.status_code == 200:
    dashboards = list_resp.json().get("dashboards", [])
    
    for dash in dashboards:
        if dash.get("display_name") == DASHBOARD_NAME:
            existing_dashboard = dash
            break
    
    if existing_dashboard:
        dashboard_id_to_delete = existing_dashboard.get("dashboard_id")
        print(f"Registered dashboard '{DASHBOARD_NAME}' found with ID: {dashboard_id_to_delete}.")
        print("Deleting existing registered dashboard...")

        delete_url = f"{host}/api/2.0/lakeview/dashboards/{dashboard_id_to_delete}"
        delete_resp = requests.delete(delete_url, headers=headers)
        
        if delete_resp.status_code in [200, 204]:
            print("Successfully deleted registered dashboard.")
        else:
            raise Exception(f"Failed to delete registered dashboard: {delete_resp.status_code} - {delete_resp.text}")
    else:
        print(f"No existing registered dashboard named '{DASHBOARD_NAME}' found.")
else:
    print(f"Warning: Could not list dashboards: {list_resp.status_code} - {list_resp.text}.")

print("---") 

# --- Create new dashboard ---
print(f"Creating new dashboard '{DASHBOARD_NAME}'...")
create_url = f"{host}/api/2.0/lakeview/dashboards"
create_payload = {
    "display_name": DASHBOARD_NAME,
    "warehouse_id": WAREHOUSE_ID
}

create_resp = requests.post(create_url, headers=headers, json=create_payload)

if create_resp.status_code not in [200, 201]:
    raise Exception(f"Dashboard creation failed: {create_resp.status_code} - {create_resp.text}")

dashboard_data = create_resp.json()
dashboard_id = dashboard_data.get("dashboard_id")
etag = dashboard_data.get("etag")
print(f"Dashboard created successfully with ID: {dashboard_id}")


# --- Read dashboard content ---
print(f"Reading dashboard definition from file...")
dashboard_json_str = dbutils.fs.head(file_path, 10000000)
dashboard_json_content = json.loads(dashboard_json_str)
print(f"Dashboard definition loaded successfully")


# --- Update the newly created dashboard with content ---
print(f"Updating dashboard {dashboard_id} with content from file...")
update_url = f"{host}/api/2.0/lakeview/dashboards/{dashboard_id}"

update_payload = {
    "display_name": DASHBOARD_NAME,
    "warehouse_id": WAREHOUSE_ID
}

# Serialize dashboard content to JSON string
if "serialized_dashboard" in dashboard_json_content:
    serialized_content = dashboard_json_content["serialized_dashboard"]
    if isinstance(serialized_content, str):
        update_payload["serialized_dashboard"] = serialized_content
    else:
        update_payload["serialized_dashboard"] = json.dumps(serialized_content)
else:
    update_payload["serialized_dashboard"] = json.dumps(dict(dashboard_json_content))

# Add etag for version control
headers_with_etag = headers.copy()
if etag:
    headers_with_etag["If-Match"] = etag

update_resp = requests.patch(update_url, headers=headers_with_etag, json=update_payload)

if update_resp.status_code != 200:
    raise Exception(f"Dashboard content update failed: {update_resp.status_code} - {update_resp.text}")

print(f"Dashboard updated successfully")
print(f"Dashboard ID: {dashboard_id}")
print(f"Dashboard URL: {host}/sql/dashboardsv3/{dashboard_id}")

current_time = time.strftime("%Y-%m-%d %H:%M:%S")
print("Process completed")
print(current_time)

In [0]:
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
dashboard_name = dbutils.widgets.get("DASHBOARD_NAME")
dashboard_id = dashboard_data.get("dashboard_id")

dbutils.notebook.exit(json.dumps({
    "dashboard_id": dashboard_id,
    "dashboard_name": dashboard_name,
    "dashboard_url": f"https://{workspace_url}/sql/dashboardsv3/{dashboard_id}"
}))